In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import csv

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier

import scikitplot.plotters as skplt

import nltk

In [2]:
test_data = pd.read_csv('data/test.csv')
train_data = pd.read_csv('data/train.csv')

/Users/sohamp/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def evaluate_features(X,y,clf=None):
    if clf is None:
        clf = LogisticRegression()
    probas = cross_val_predict(clf, X, y, cv=StratifiedKFold(random_state=8), 
                              n_jobs=-1, method='predict_proba', verbose=2)
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    skplt.plot_confusion_matrix(y, preds)

In [4]:
vectorizer = CountVectorizer(analyzer="word",tokenizer=nltk.word_tokenize,preprocessor=None,
                                  stop_words='english',max_features=None)
# vectorizer = TfidfVectorizer(analyzer="word", tokenizer=nltk.word_tokenize,preprocessor=None, stop_words='english',max_features=None)


In [6]:
all_questions = pd.concat((train_data['question1'],train_data['question2']),ignore_index=True)
bag_of_words = vectorizer.fit_transform(all_questions.values.astype('U'))

In [7]:
svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
truncated_bag_of_words = svd.fit_transform(bag_of_words)
truncated_bag_of_words = np.reshape(truncated_bag_of_words,(int(np.shape(truncated_bag_of_words)[0]/2),50))

In [ ]:
evaluate_features(truncated_bag_words,train_data['is_duplicate'].values.ravel() )
'''
evaluate_features(truncated_bag_of_words[:20000], train_data['is_duplicate'].values.ravel(), 
                  SVC(kernel='linear', probability=True))
evaluate_features(truncated_bag_of_words, train_data['is_duplicate'].values.ravel(), RandomForestClassifier(n_estimators=1000, max_depth=5, verbose=1))
'''